<a href="https://colab.research.google.com/github/DataScienceAndEngineering/deep-learning-final-project-project-sidewalk/blob/rabiul/notebooks/Rabiul/Cityscapes_data_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# Importing Lib

In [ ]:
#
%%time
import numpy as np
import pandas as pd
import os
import PIL
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm.notebook import tqdm
from PIL import Image

from os.path import join, isdir
from os import listdir, rmdir
from shutil import move, rmtree, make_archive

import cv2
import glob
import pickle

from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, Dropout, Activation, Flatten, BatchNormalization, ReLU, LeakyReLU, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, GlobalAveragePooling2D, Add

In [ ]:
%%time
#drive.mount('/gdrive')
drive_root = '/content/drive/MyDrive/DL_Project/'


COLAB_DIR = '/content/'
GT_DIR = COLAB_DIR + 'gtFine/'
IMG_DIR = COLAB_DIR + 'leftImg8bit/'

# Initial Data processing

In [ ]:
%%time
!gdown --id 1mIDcIT_7vP9cQDl_D8zgX9g4Pe18-g5q -O leftImg8bit_trainvaltest.zip
!gdown --id 1ExL_qMIykksn_kpxZxi5DvltBgX-gHVn -O gtFine_trainvaltest.zip

In [ ]:
%%time
!unzip /content/drive/MyDrive/DL_Project/Data/gtFine_trainvaltest.zip
!unzip /content/drive/MyDrive/DL_Project/Data/leftImg8bit_trainvaltest.zip

In [ ]:
%%time
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

In [ ]:
%%time
# collapse child directories
for parent in listdir(GT_DIR):
    parent_dir = GT_DIR + parent
    for child in listdir(parent_dir):
        if isdir(join(parent_dir, child)):
            keep = glob.glob(join(parent_dir, child) + '/*_gtFine_color.png')
            keep = [f.split('/')[-1] for f in keep]
            for filename in list(set(listdir(join(parent_dir, child))) & set(keep)):
                move(join(parent_dir, child, filename), join(parent_dir, filename))
            rmtree(join(parent_dir, child))

for parent in listdir(IMG_DIR):
    parent_dir = IMG_DIR + parent
    for child in listdir(parent_dir):
        if isdir(join(parent_dir, child)):
            for filename in listdir(join(parent_dir, child)):
                move(join(parent_dir, child, filename), join(parent_dir, filename))
            rmtree(join(parent_dir, child))

In [ ]:
%%time
# process anr archive image in smaller size
IMG_SHAPE = 299, 299

gt_train_paths = [GT_DIR+'train/' + path for path in listdir(GT_DIR+'train/')]
gt_test_paths = [GT_DIR+'test/' + path for path in listdir(GT_DIR+'test/')]
gt_val_paths = [GT_DIR+'val/' + path for path in listdir(GT_DIR+'val/')]
gt_paths = gt_train_paths + gt_test_paths + gt_val_paths

im_train_paths = [IMG_DIR+'train/' + path for path in listdir(IMG_DIR+'train/')]
im_test_paths = [IMG_DIR+'test/' + path for path in listdir(IMG_DIR+'test/')]
im_val_paths = [IMG_DIR+'val/' + path for path in listdir(IMG_DIR+'val/')]
im_paths = im_train_paths + im_test_paths + im_val_paths

def resize_image(path):
    img = Image.open(path)
    img.thumbnail(IMG_SHAPE)
    out_file = join(path)
    img.save(out_file, 'PNG')

for img in gt_paths + im_paths:
    resize_image(img)
#saving the data for future use, avoding pre_processing task over and over 
make_archive('gtFine', 'zip', GT_DIR) 
make_archive('leftImg', 'zip', IMG_DIR)

# Re-Load Pre_processed Data

In [ ]:
%%time
#PreProcessed data is located in my personal Google Drive, its publically available. anyone can download it. 
!gdown --id 1kkmpZOGdncxjI6NEvFmZh85Wn2nrW3rK -O gtFine.zip
!gdown --id 14MXcx8G3yRLljLb1K0PYX7yCl5viPP-8 -O leftImg.zip


In [ ]:
%%time
!unzip /content/gtFine.zip -d '/content/gtFine/'
!unzip /content/leftImg.zip -d '/content/leftImg/'

In [ ]:
# normalize image pixels
%%time
IMG_SIZE = 299
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

GT_DIR = COLAB_DIR + 'gtFine/'
IMG_DIR = COLAB_DIR + 'leftImg/'

def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img /= 255.0
    return img

def get_image_paths(dir):
    return sorted([dir + path for path in listdir(dir)])

# create tf.Dataset objects
gt_train_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(GT_DIR+'train/'))
gt_val_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(GT_DIR+'val/'))
gt_test_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(GT_DIR+'test/'))

gt_train_ds = gt_train_ds.map(load_and_preprocess_image)
gt_val_ds = gt_val_ds.map(load_and_preprocess_image)
gt_test_ds = gt_test_ds.map(load_and_preprocess_image)

im_train_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'train/'))
im_val_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'val/'))
im_test_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'test/'))

im_train_ds = im_train_ds.map(load_and_preprocess_image)
im_val_ds = im_val_ds.map(load_and_preprocess_image)
im_test_ds = im_test_ds.map(load_and_preprocess_image)

# Visualize Data

In [ ]:
def visualize_images(img, gt, pred):
    if pred is not None:
        fig, axes = plt.subplots(1, 3, figsize=(12, 8))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(8, 8))

    axes[0].imshow(img)
    axes[0].set_title('Actual Image')

    axes[1].imshow(gt)
    axes[1].set_title('Masked Image')
    
    if pred is not None:
        axes[2].imshow(pred)
        axes[2].set_title('Predicted Image')

In [ ]:
%%time
for img, gt in list(zip(im_train_ds.take(2), gt_train_ds.take(2))):
    visualize_images(img, gt, None)

#Model

In [ ]:
def conv_block(X,filters,block):
    # resiudal block with dilated convolutions
    # add skip connection at last after doing convoluion

    b = 'block_'+str(block)+'_'
    f1,f2,f3 = filters
    X_skip = X

    # block_a
    X = Conv2D(filters=f1,kernel_size=(1,1),dilation_rate=(1,1),
               padding='same',kernel_initializer='he_normal',name=b+'a')(X)
    X = BatchNormalization(name=b+'batch_norm_a')(X)
    X = LeakyReLU(alpha=0.2,name=b+'leakyrelu_a')(X)
    # block_b
    X = Conv2D(filters=f2,kernel_size=(3,3),dilation_rate=(2,2),
               padding='same',kernel_initializer='he_normal',name=b+'b')(X)
    X = BatchNormalization(name=b+'batch_norm_b')(X)
    X = LeakyReLU(alpha=0.2,name=b+'leakyrelu_b')(X)
    # block_c
    X = Conv2D(filters=f3,kernel_size=(1,1),dilation_rate=(1,1),
               padding='same',kernel_initializer='he_normal',name=b+'c')(X)
    X = BatchNormalization(name=b+'batch_norm_c')(X)
    # skip_conv
    X_skip = Conv2D(filters=f3,kernel_size=(3,3),padding='same',name=b+'skip_conv')(X_skip)
    X_skip = BatchNormalization(name=b+'batch_norm_skip_conv')(X_skip)
    # block_c + skip_conv
    X = Add(name=b+'add')([X,X_skip])
    X = ReLU(name=b+'relu')(X)
    return X
    
def base_feature_maps(input_layer):
    # base covolution module to get input image feature maps 
    
    # block_1
    base = conv_block(input_layer,[16,16,32],'1')
    # block_2
    base = conv_block(base,[16,16,32],'2')
    return base

def pyramid_feature_maps(input_layer):
    # pyramid pooling module
    
    base = base_feature_maps(input_layer)
    # red
    red = GlobalAveragePooling2D(name='red_pool')(base)
    red = tf.keras.layers.Reshape((1,1,32))(red)
    red = Conv2D(filters=32,kernel_size=(1,1),name='red_1_by_1')(red)
    red = UpSampling2D(size=128,interpolation='bilinear',name='red_upsampling')(red)
    red = tf.image.resize(red, [IMG_SIZE, IMG_SIZE])
    # yellow
    yellow = AveragePooling2D(pool_size=(2,2),name='yellow_pool')(base)
    yellow = Conv2D(filters=32,kernel_size=(1,1),name='yellow_1_by_1')(yellow)
    yellow = UpSampling2D(size=2,interpolation='bilinear',name='yellow_upsampling')(yellow)
    yellow = tf.image.resize(yellow, [IMG_SIZE, IMG_SIZE])
    # blue
    blue = AveragePooling2D(pool_size=(4,4),name='blue_pool')(base)
    blue = Conv2D(filters=32,kernel_size=(1,1),name='blue_1_by_1')(blue)
    blue = UpSampling2D(size=4,interpolation='bilinear',name='blue_upsampling')(blue)
    blue = tf.image.resize(blue, [IMG_SIZE, IMG_SIZE])
    # green
    green = AveragePooling2D(pool_size=(8,8),name='green_pool')(base)
    green = Conv2D(filters=32,kernel_size=(1,1),name='green_1_by_1')(green)
    green = UpSampling2D(size=8,interpolation='bilinear',name='green_upsampling')(green)
    green = tf.image.resize(green, [IMG_SIZE, IMG_SIZE])
    # base + red + yellow + blue + green
    return tf.keras.layers.concatenate([base,red,yellow,blue,green])

def last_conv_module(input_layer):
    X = pyramid_feature_maps(input_layer)
    X = Conv2D(filters=3,kernel_size=3,padding='same',name='last_conv_3_by_3')(X)
    X = BatchNormalization(name='last_conv_3_by_3_batch_norm')(X)
    X = Activation('sigmoid',name='last_conv_relu')(X)
    return X

In [ ]:
%%time
input_shape = list(im_train_ds.take(1))[0].shape
input_layer = tf.keras.Input(shape=input_shape, name='input')
output_layer = last_conv_module(input_layer)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Train

In [ ]:
%%time
train_ds = tf.data.Dataset.zip((im_train_ds, gt_train_ds))
train_ds = train_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

val_ds = tf.data.Dataset.zip((im_val_ds, gt_val_ds))
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

test_ds = tf.data.Dataset.zip((im_test_ds, gt_test_ds))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

In [ ]:
%%time
checkpoint_path = drive_root+'pspnet/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model.compile(optimizer='adam', loss='mse', metrics='accuracy')
history = model.fit(train_ds, validation_data=val_ds, epochs=5, 
                    callbacks=[cp_callback, es_callback])
model.save(drive_root + 'pspnet_trained.h5')

In [ ]:
def plot(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1,len(acc)+1)

  plt.title('Training and validation accuracy')
  plt.plot(epochs, acc, color='blue', label='Train')
  plt.plot(epochs, val_acc, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

  _ = plt.figure()
  plt.title('Training and validation loss')
  plt.plot(epochs, loss, color='blue', label='Train')
  plt.plot(epochs, val_loss, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  
plot(history)

In [ ]:
loss, acc = model.evaluate(test_ds)
print('Test Data Accuracy: ', acc)

In [ ]:
pred_test_ds = model.predict(train_ds)

for img, gt, pred in list(zip(im_train_ds.take(5), gt_train_ds.take(5), pred_test_ds)):
    visualize_images(img, gt, pred)

# Unet

In [ ]:
%%time
# normalize image pixels
IMG_SIZE1 =200
IMG_SIZE2 =256
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

GT_DIR = COLAB_DIR + 'gtFine/'
IMG_DIR = COLAB_DIR + 'leftImg/'
def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE1, IMG_SIZE2])
    img /= 255.0
    return img

def get_image_paths(dir):
    return sorted([dir + path for path in listdir(dir)])

# data transformation using K-means
def LayersToRGBImage(img):
    colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255), 
              (0,255,255), (255,255,255), (200,50,0),(50,200,0), (50,0,200), 
              (200,200,50), (0,50,200), (0,200,50), (0,0,0)]
    
    nimg = np.zeros((img.shape[0], img.shape[1], 3))
    for i in range(img.shape[2]):
        c = img[:,:,i]
        col = colors[i]
        
        for j in range(3):
            nimg[:,:,j]+=col[j]*c
    nimg = nimg/255.0
    return nimg
    
def train_kmeans(K=7):
    colors = []
    kmeans_gt_train_paths = [GT_DIR + 'train/' + path for path in listdir(GT_DIR + 'train/')[:20]]
    kmeans_gt_train_ds = tf.data.Dataset.from_tensor_slices(kmeans_gt_train_paths)
    kmeans_gt_train_ds = kmeans_gt_train_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

    for seg in kmeans_gt_train_ds:
        colors.append(tf.reshape(seg, (seg.shape[0]*seg.shape[1], 3) ))
    colors = tf.concat(colors, 0) 
    print("Training K means on colors data of shape :",colors.shape)
    km = KMeans(K)
    km.fit(colors)
    print("\nK-means clustering trained\n", km)
    return km

# Traing a k-means classifier
km=train_kmeans(13)

def load_and_preprocess_segment(path):
    seg = load_and_preprocess_image(path)
    s = tf.reshape(seg, (seg.shape[0]*seg.shape[1],3))
    s = km.predict(s)
    s = tf.reshape(s, (seg.shape[0], seg.shape[1]))
    n = len(km.cluster_centers_)
    cls = np.zeros((seg.shape[0], seg.shape[1], n))
    for i in range(n):
        m = np.copy(s)
        m[m!=i] = 0
        m[m!=0] = 1
        cls[:,:,i]=m

    seg = tf.convert_to_tensor(cls, dtype=tf.float32)
    return seg

In [ ]:
%%time
# train data
im_train_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'train/'))
im_train_ds = im_train_ds.map(load_and_preprocess_image)
gt_train_paths = get_image_paths(GT_DIR+'train/')
gt_train_ds = map(load_and_preprocess_segment, gt_train_paths)

In [ ]:
%%time
gt_train_ds = tf.data.Dataset.from_tensor_slices([a for a in gt_train_ds])

In [ ]:
%%time
# train data
im_train_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'train/'))
im_train_ds = im_train_ds.map(load_and_preprocess_image)

gt_train_paths = get_image_paths(GT_DIR+'train/')
gt_train_ds = map(load_and_preprocess_segment, gt_train_paths)
gt_train_ds = tf.data.Dataset.from_tensor_slices([a for a in gt_train_ds])
train_ds = tf.data.Dataset.zip((im_train_ds, gt_train_ds))
train_ds = train_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

print('Training Data:\n# of batches, Input batch shape, Ouput batch shape')
print(len(train_ds), next(iter(train_ds))[0].shape, next(iter(train_ds))[1].shape)

In [ ]:
%%time
# val data
im_val_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'val/'))
im_val_ds = im_val_ds.map(load_and_preprocess_image)

gt_val_paths = get_image_paths(GT_DIR+'val/')
gt_val_ds = map(load_and_preprocess_segment, gt_val_paths)
gt_val_ds = tf.data.Dataset.from_tensor_slices([a for a in gt_val_ds])
val_ds = tf.data.Dataset.zip((im_val_ds, gt_val_ds))
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)


print('Validation Data:\n# of batches, Input batch shape, Ouput batch shape')
print(len(val_ds), next(iter(val_ds))[0].shape, next(iter(val_ds))[1].shape)

In [ ]:
%%time
# test data
im_test_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR+'test/'))
im_test_ds = im_test_ds.map(load_and_preprocess_image)

gt_test_paths = get_image_paths(GT_DIR+'test/')
gt_test_ds = map(load_and_preprocess_segment, gt_test_paths)
gt_test_ds = tf.data.Dataset.from_tensor_slices([a for a in gt_test_ds])
test_ds = tf.data.Dataset.zip((im_test_ds, gt_test_ds))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

print('Testing Data:\n# of batches, Input batch shape, Ouput batch shape')
print(len(test_ds), next(iter(test_ds))[0].shape, next(iter(test_ds))[1].shape)

In [ ]:
%%time

# Baseline Model

## Loading Lib

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns
import pandas as pd
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd

##  Loading Data

In [ ]:
%%time
!gdown --id 12rNfOiM4allOn8t4l0ErpvgmA9hEZPh2 -O processed_labels.zip
!gdown --id 145Ul0rdbQcX98lQz8IO5eOtI0ZlvKwz6 -O processed_images.zip

In [ ]:
%%time
!unzip /content/processed_images.zip
!unzip /content/processed_labels.zip

### Pre-Processing Data

In [ ]:
img_path = '/content/content/processed_images/*'
lab_path = '/content/content/processed_labels/*'

In [ ]:
#loading the data indo a list, numpy array
%%time
train_images = glob.glob(img_path)
train_labels = glob.glob(lab_path)

print(len(train_images))
print(len(train_labels))

In [ ]:
img = train_images[0]
lab = train_labels[0]
print(img)
print(lab)

In [ ]:
img = cv2.imread(img)
print(img, img.shape)

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(gray)

In [ ]:
gimg1 = gabor_feature_extraction(img,.8, .05 )
gimg2 = gabor_feature_extraction(img,1.6, .5 )

In [ ]:
plt.imshow(gimg2.reshape(img.shape))


In [ ]:
plt.imshow(gimg1.reshape(img.shape))

In [ ]:
plt.imshow(img)

In [ ]:
blur1, blur2, sobele, canny = preprocess_image(img)
print(blur1.shape, blur2.shape, sobele.shape, canny.shape )

In [ ]:
plt.imshow(blur1.reshape(img.shape))

In [ ]:
plt.imshow(blur2.reshape(img.shape))

In [ ]:
plt.imshow(sobele.reshape(img.shape))

In [ ]:
plt.imshow(canny.reshape(img.shape))

# WIP

In [ ]:
gray1 = gray.reshape(-1,1)
gray1.shape

In [ ]:
gabor1 = gabor_feature_extraction(gray,.8, .05 )
gabor2 = gabor_feature_extraction(gray,1.6, .5 )
gabor1.shape, gabor2.shape

In [ ]:
blur1, blur2, sobele, canny = preprocess_image(gray)
print(blur1.shape, blur2.shape, sobele.shape, canny.shape )

In [ ]:
    r, g, b = cv2.split(img)
    features = np.dstack((r, g, b)).reshape(-1, 3)

In [ ]:
features = np.hstack((gray1, gabor1, gabor2, blur1, blur2, sobele, canny))
features.shape

In [ ]:
for directory_path in glob.glob("images/natural/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR) #Reading color images
        img = cv2.resize(img, (SIZE, SIZE)) #Resize images
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #Optional step. Change BGR to RGB
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
#Now, add a column in the data frame for the Labels
#For this, we need to import the labeled image
labeled_img = cv2.imread(lab)
#Remember that you can load an image with partial labels 
#But, drop the rows with unlabeled data

labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
labeled_img1 = labeled_img.reshape(-1)


In [ ]:
labeled_img1.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(labeled_img1)

In [ ]:
Y.shape

In [ ]:
)

In [ ]:
original_img_data = df.drop(labels = ["Labels"], axis=1) #Use for prediction
#df.to_csv("Gabor.csv")
df = df[df.Labels != 0]

In [ ]:
#Define the dependent variable that needs to be predicted (labels)
Y = df["Labels"].values

#Encode Y values to 0, 1, 2, 3, .... (NOt necessary but makes it easy to use other tools like ROC plots)
from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(Y)


#Define the independent variables
X = df.drop(labels = ["Labels"], axis=1) 

In [ ]:
#Split data into train and test to verify accuracy after fitting the model. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)



In [ ]:
features.shape, Y.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with n number of decision trees
model = RandomForestClassifier(n_estimators = 20, random_state = 42)

# Train the model on training data
model.fit(features, Y)

In [ ]:
feature_list = list(X.columns)
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
print(feature_imp)

In [ ]:
prediction_RF = model.predict(features)

In [ ]:
prediction_RF.shape

In [ ]:
plt.imshow(prediction_RF.reshape(gray.shape))

In [ ]:
lab = cv2.imread('/content/content/processed_labels/aachen_000159_000019.png',0)
print(lab.shape)

In [ ]:
plt.imshow(lab)

In [ ]:
img1 = train_images[11]
lab1 = train_labels[11]
lab1 = cv2.imread(lab1,0)
img1 = cv2.imread(img1,0)
print(lab1.shape, img1.shape)

In [ ]:
plt.imshow(img1)

In [ ]:
plt.imshow(lab1)

# Evl

In [ ]:
gabor1 = gabor_feature_extraction(img1,.8, .05 )
gabor2 = gabor_feature_extraction(img1,1.6, .5 )
blur1, blur2, sobele, canny = preprocess_image(img1)
print(blur1.shape, blur2.shape, sobele.shape, canny.shape )
print(gabor1.shape, gabor2.shape)

In [ ]:
features = np.hstack((gray1, gabor1, gabor2, blur1, blur2, sobele, canny))
features.shape

In [ ]:
lab1.shape

In [ ]:
prediction_RF = model.predict(features)

In [ ]:
plt.imshow(prediction_RF.reshape(gray.shape))

In [ ]:
plt.imshow(lab1)

In [ ]:
from sklearn import metrics
#Print the prediction accuracy
#Check accuracy on test dataset. If this is too low compared to train it indicates overfitting on training data.
print ("Accuracy using Random Forest= ", metrics.accuracy_score(y_test, prediction_RF))

# new approach

In [ ]:
/content/content/processed_images/dusseldorf_000143_000019.png
/content/content/processed_labels/dusseldorf_000143_000019.png

In [ ]:
img, lab

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier



In [ ]:
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(edges, (5, 5), 0)
    
    return blur

In [ ]:
def npreprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).reshape(-1,1)
    sobels = sobel(gray)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(edges, (5, 5), 0)
    
    return np.hstack((gray, sobels,edges,blur))

In [ ]:
def extract_features(img):
    # Extract RGB color features
    r, g, b = cv2.split(img)
    features = np.dstack((r, g, b)).reshape(-1, 3)
    
    # Extract texture features using Gabor filter
    kernel = cv2.getGaborKernel((5, 5), 4, np.pi/4, 8, 0.5, 0, ktype=cv2.CV_32F)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    filtered = cv2.filter2D(gray, cv2.CV_8UC3, kernel)
    texture_features = filtered.reshape(-1, 1)
    
    # Combine color and texture features
    features = np.hstack((features, texture_features))
    
    return features

In [ ]:
def segment_image(img):
    # Preprocess the image
    preprocessed = preprocess_image(img)
    
    # Extract features from the preprocessed image
    features = extract_features(img)
    
    # Cluster the features using k-means
    kmeans = KMeans(n_clusters=3, random_state=0).fit(features)
    labels = kmeans.labels_.reshape(preprocessed.shape[:2])
    
    # Train a Random Forest classifier on the labeled features
    flat_features = features.reshape(-1, features.shape[-1])
    clf = RandomForestClassifier(n_estimators=50, random_state=0)
    clf.fit(flat_features, labels.ravel())
    
    # Predict the segmentation for the preprocessed image
    flat_preprocessed = preprocessed.reshape(-1, preprocessed.shape[-1])
    segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])
    
    # Convert the segmentation to a binary mask
    mask = np.zeros_like(segmentation)
    mask[segmentation == 1] = 255
    
    return mask

In [ ]:
# Load an example image
img = cv2.imread('/content/content/processed_images/dusseldorf_000143_000019.png')

In [ ]:

# Preprocess the image
preprocessed = preprocess_image(img)
npreprocessed = npreprocess_image(img)
print(preprocessed.shape, npreprocessed.shape)

In [ ]:
# Extract features from the preprocessed image
features = extract_features(img)
features.shape

In [ ]:
# Cluster the features using k-means
kmeans = KMeans(n_clusters=21, random_state=0).fit(features)
labels = kmeans.labels_.reshape(preprocessed.shape[:2])
labels.shape

In [ ]:
preprocessed.shape[:2]

In [ ]:
# Cluster the features using k-means
kmeans = KMeans(n_clusters=21, random_state=0).fit(features)
labels = kmeans.labels_.reshape(npreprocessed.shape[:2])
labels.shape

In [ ]:
labels[10]

In [ ]:
flat_features = features.reshape(-1, features.shape[-1])
print(flat_features.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=50, random_state=0)

In [ ]:
labels.ravel().shape

In [ ]:
clf.fit(flat_features, labels.ravel())

flat_features = features.reshape(-1, features.shape[-1])
print(flat_features.shape)

In [ ]:
# Predict the segmentation for the preprocessed image
#flat_preprocessed = preprocessed.reshape(-1, preprocessed.shape[-1])
flat_preprocessed = preprocessed.reshape(-1, preprocessed.shape[-1])
flat_preprocessed.shape

In [ ]:
#segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])   
segmentation = clf.predict(npreprocessed)

segmentation.shape

In [ ]:
segmentation = segmentation.reshape(preprocessed.shape[:2]) 
segmentation.shape

In [ ]:
plt.imshow(segmentation)

In [ ]:
# Convert the segmentation to a binary mask
mask = np.zeros_like(segmentation)
mask[segmentation == 1] = 255

In [ ]:


segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])

# Convert the segmentation to a binary mask
mask = np.zeros_like(segmentation)
mask[segmentation == 1] = 255


In [ ]:
# Segment the image
##mask = segment_image(img)



In [ ]:
mask.shape, img.shape

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(mask)

In [ ]:
# Display the results
cv2.imshow('Input Image', img)
cv2.imshow('Segmentation Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).reshape(-1,1)
    sobels = sobel(gray)
    
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(edges, (5, 5), 0)
    
    return np.hstack((gray, sobels,edges,blur))

In [ ]:
a = preprocess_image(img)


In [ ]:
a.shape

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray.shape
g= gray.reshape(-1, 1)
g.shape

In [ ]:
#SOBEL
s = sobel(g)
#s = s.reshape(-1,1)
s.shape

In [ ]:
 # Apply Canny edge detection
edges = cv2.Canny(gray, 100, 200)
edges.shape
e= edges.reshape(-1,1)

In [ ]:
blur = cv2.GaussianBlur(edges, (5, 5), 0)
blur.shape
b= blur.reshape(-1,1)

In [ ]:
blur.reshape(-1).shape

In [ ]:
blur.shape

In [ ]:
f = np.hstack((g, e,b,s))
f.shape

In [ ]:
segmentation = clf.predict(f).reshape(preprocessed.shape[:2])
    

In [ ]:
mask = np.zeros_like(segmentation)
mask[segmentation == 1] = 255

In [ ]:
plt.imshow(mask)

In [ ]:
lab = train_label[0]
lab = cv2.imread(lab, 0)


In [ ]:
plt.imshow(lab)


In [ ]:
    segmentation = clf.predict(flat_preprocessed).reshape(preprocessed.shape[:2])
    
    # Convert the segmentation to a binary mask
    mask = np.zeros_like(segmentation)
    mask[segmentation == 1] = 255
    
    return mask

# Load an example image
img = cv2.imread('example.jpg')

# Segment the image
mask = segment_image(img)

# Display the results
cv2.imshow('Input Image', img)
cv2.imshow('Segmentation Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Functions

In [ ]:
def gabor_feature_extraction(img, lamda, gamma):
  kernel = cv2.getGaborKernel((8, 8), 1, 0, lamda, gamma, 0, ktype=cv2.CV_32F)
  gabor_img = cv2.filter2D(img, cv2.CV_8UC3, kernel)
  return gabor_img.reshape(-1,1)


In [ ]:
def preprocess_image(img):
  #GAUSSIAN blur 1
  blur1 = nd.gaussian_filter(img, sigma=3)
  #GAUSSIAN blur2
  blur2 = nd.gaussian_filter(img, sigma=7)
  #SOBEL
  sobele = sobel(blur2)
  #CANNY 
  cany = cv2.Canny(np.uint8(sobele), 100,200)
  #return cany
  return (blur1.reshape(-1),blur2.reshape(-1),sobele.reshape(-1),cany.reshape(-1))

In [ ]:
def preprocess_image_org(img):
  #GAUSSIAN blur 1
  blur1 = nd.gaussian_filter(img, sigma=3)
  #GAUSSIAN blur2
  blur2 = nd.gaussian_filter(img, sigma=7)
  #SOBEL
  sobele = sobel(blur2)
  #CANNY 
  cany = cv2.Canny(np.uint8(sobele), 100,200)
  #return cany
  return (blur1.reshape(-1,1),blur2.reshape(-1,1),sobele.reshape(-1,1),cany.reshape(-1,1))

In [ ]:
def display_img(img):
  fig = plt.figure(figsize=(15,15))
  ax = fig.add_subplot(111)
  ax.imshow(img,cmap='gray')

In [ ]:
def load_img(image_path):
  return cv2.imread(image_path, 0).astype(np.float32)/255


#New Approach

In [ ]:
image_dataset = pd.DataFrame() 

In [ ]:
train_images[200]

In [ ]:
#train_images = glob.glob(img_path)
#train_labels = glob.glob(lab_path)
print(len(train_images))
print(len(train_labels))
print(int(len(train_images)*.7))

In [ ]:
m = cv2.imread(train_images[200], 0)

In [ ]:
plt.imshow(m, cmap='gray')

In [ ]:
#Thresholding
ret,thresh1  = cv2.threshold(m, 125, 255, cv2.THRESH_BINARY)
plt.imshow(thresh1, cmap='gray')

In [ ]:
#Thresholding
ret,thresh2  = cv2.threshold(m, 125, 255, cv2.THRESH_BINARY_INV)
plt.imshow(thresh2, cmap='gray')

In [ ]:
display_img(thresh2)

In [ ]:
th2 = cv2.adaptiveThreshold(m, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 8)
display_img(th2)

In [ ]:
ret,thresh2  = cv2.threshold(m, 125, 255, cv2.THRESH_BINARY_INV)

In [ ]:
m1 = load_img(train_images[200])
display_img(m1)

In [ ]:
gm = np.power(m1,4)
display_img(gm)

In [ ]:
cv2blur = cv2.blur(m1, ksize=(5,5))
display_img(cv2blur)

In [ ]:
gblur = cv2.GaussianBlur(m1, (5,5),8)
display_img(gblur)

In [ ]:
mblur = cv2.medianBlur(m1,5)
display_img(mblur)

In [ ]:
s1 = cv2.Sobel(m1,cv2.CV_64F, 1,0,ksize=5)
display_img(s1)

In [ ]:
cany = cv2.Canny(np.uint8(sobele), 100,200)

In [ ]:
c1 = cv2.Canny(np.uint8(s1), 100,200)
display_img(c1)

In [ ]:
laplacian = cv2.Laplacian(m1, cv2.CV_64F, ksize=7) 
display_img(laplacian)

In [ ]:
laplacian.shape

In [ ]:
def preprocess_image(image_dir):
  #loading the image and normalize it 
  m1 = load_img(image_dir)

  #GAUSSIAN blur 1
  gblur = cv2.GaussianBlur(m1, (5,5),7)
  #Laplacian
  laplacian = cv2.Laplacian(m1, cv2.CV_64F, ksize=7) 
  #SOBEL
  sobele = cv2.Sobel(m1,cv2.CV_64F, 0,1,ksize=5)
  #CANNY 
  cany = cv2.Canny(np.uint8(s1), 100,200)

  #return cany
  return (gblur.reshape(-1),laplacian.reshape(-1),sobele.reshape(-1),cany.reshape(-1))

In [ ]:
b = nd.gaussian_filter(m, sigma=3)
plt.imshow(b)

In [ ]:

#SOBEL
s = sobel(b)
plt.imshow(s)


In [ ]:
  #CANNY 
c= cv2.Canny(np.uint8(s), 100,200)
plt.imshow(c,cmap='gray')

In [ ]:
np.max(c,cmap='gray')

In [ ]:
#create a data frame with test data image 
df_image = pd.DataFrame() 
for image in train_images[:200]:
  df1 = pd.DataFrame()
  img = cv2.imread(image, 0)
  df1['gray']= img.reshape(-1)
  df1['Image_Name'] = image.split('/')[-1]
  df1['gabor1'] = gabor_feature_extraction(img,.8, .05 ).reshape(-1)
  df1['gabor2'] = gabor_feature_extraction(img,1.6, .5 ).reshape(-1)
  df1['blur1'], df1['blur2'], df1['sobelx'], df1['canny'] = preprocess_image(img)
  df_image = pd.concat([df_image, df1])

print(df_image.shape)

In [ ]:
#Creating test mask image Data Frame
%%time
df_mask = pd.DataFrame() 
for mask in train_labels[:100]:
  df2 = pd.DataFrame()
  msk = cv2.imread(mask, 0)
  df2['label'] = msk.reshape(-1) 
  df2['Mask_Name'] = mask.split('/')[-1]  
  df_mask = pd.concat([df_mask, df2])

print(df_mask.shape)

In [ ]:
df_mask.shape

In [ ]:
df_image.shape

In [ ]:
df_image.head()

In [ ]:
df_mask.head()

In [ ]:
df = pd.concat([df_image, df_mask], axis=1)  

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df_0 = df[df.label != 0]
df_0.shape

In [ ]:
X = df_0.drop(labels = ["Image_Name", "Mask_Name", "label"], axis=1) 

In [ ]:
Y = df_0["label"].values 

In [ ]:
X.shape, Y.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(Y)
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

In [ ]:
#Import training classifier
from sklearn.ensemble import RandomForestClassifier
## Instantiate model with n number of decision trees
model = RandomForestClassifier(n_estimators = 50, random_state = 42)



In [ ]:
## Train the model on training data
model.fit(X, Y)

In [ ]:
test_img = cv2.imread(train_images[100], 0)
test_lab = cv2.imread(train_labels[100],0)

In [ ]:
plt.imshow(test_img)

In [ ]:
plt.imshow(test_lab)

In [ ]:
gabor1 = gabor_feature_extraction(test_img,.8, .05 )
gabor2 = gabor_feature_extraction(test_img,1.6, .5 )
blur1, blur2, sobele, canny = preprocess_image(test_img)
print(blur1.shape, blur2.shape, sobele.shape, canny.shape )
print(gabor1.shape, gabor2.shape)

In [ ]:
gray1=test_img.reshape(-1,1)
features = np.hstack((gray1, gabor1, gabor2, blur1, blur2, sobele, canny))
features.shape

In [ ]:
blur1.shape

In [ ]:
type(X)

In [ ]:
test_df = pd.DataFrame()
test_df['gray']= test_img.reshape(-1)#img.reshape(-1)
#df1['Image_Name'] = image.split('/')[-1]  
test_df['gabor1'] = gabor_feature_extraction(test_img,.8, .05 ).reshape(-1)
test_df['gabor2'] = gabor_feature_extraction(test_img,1.6, .5 ).reshape(-1)
test_df['blur1'], test_df['blur2'], test_df['sobelx'], test_df['canny'] = preprocess_image(test_img)

In [ ]:
test_df.shape

In [ ]:
from sklearn import metrics
prediction_test = model.predict(test_df)

In [ ]:
test_lab.reshape(-1).shape

In [ ]:
prediction_test.shape

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_lab.reshape(-1), prediction_test))

In [ ]:
plt.imshow(test_lab)

In [ ]:
plt.imshow(prediction_test.reshape(test_lab.shape))

In [ ]:
test_lab.shape

In [ ]:
prediction_test.shape

In [ ]:
X_test.shape